# Word2Vec
As described in [A Word2Vec Keras tutorial](https://adventuresinmachinelearning.com/word2vec-keras-tutorial/)


![](https://i0.wp.com/adventuresinmachinelearning.com/wp-content/uploads/2017/08/Negative-sampling-architecture-1.jpg)

In [1]:
from keras.models import Model
from keras.layers import Input, Dense, Reshape, Lambda, dot
from keras.layers.embeddings import Embedding
from keras.preprocessing.sequence import skipgrams
from keras.preprocessing import sequence

import urllib
import urllib.request
import collections
import os
import zipfile

import numpy as np
import tensorflow as tf


Using TensorFlow backend.


In [2]:
def maybe_download(filename, url, expected_bytes):
    """Download a file if not present, and make sure it's the right size."""
    if not os.path.exists(filename):
        print("ARGHHH error")
        filename, _ = urllib.request.urlretrieve(url + filename, filename)
    statinfo = os.stat(filename)
    if statinfo.st_size == expected_bytes:
        print('Found and verified', filename)
    else:
        print(statinfo.st_size)
        raise Exception(
            'Failed to verify ' + filename + '. Can you get to it with a browser?')
    return filename


# Read the data into a list of strings.
def read_data(filename):
    """Extract the first file enclosed in a zip file as a list of words."""
    with zipfile.ZipFile(filename) as f:
        data = tf.compat.as_str(f.read(f.namelist()[0])).split()
    return data

In [3]:
https://www.youtube.com/watch?v=JILfse0prtw
# Read the data into a list of strings.
def read_data(filename):
    """Extract the first file enclosed in a zip file as a list of words."""
    with zipfile.ZipFile(filename) as f:
        data = tf.compat.as_str(f.read(f.namelist()[0])).split()
    return data


def build_dataset(words, n_words):
    """Process raw inputs into a dataset."""
    count = [['UNK', -1]]
    count.extend(collections.Counter(words).most_common(n_words - 1))
    dictionary = dict()
    for word, _ in count:
        dictionary[word] = len(dictionary)
    data = list()
    unk_count = 0
    for word in words:
        if word in dictionary:
            index = dictionary[word]
        else:
            index = 0  # dictionary['UNK']
            unk_count += 1
        data.append(index)
    count[0][1] = unk_count
    reversed_dictionary = dict(zip(dictionary.values(), dictionary.keys()))
    return data, count, dictionary, reversed_dictionary

def collect_data(vocabulary_size=10000):
    url = 'http://mattmahoney.net/dc/'
    #filename = maybe_download('text8.zip', url, 31344016)
    vocabulary = read_data("data1.zip")
    print(vocabulary[:7])
    data, count, dictionary, reverse_dictionary = build_dataset(vocabulary,
                                                                vocabulary_size)
    del vocabulary  # Hint to reduce memory.
    return data, count, dictionary, reverse_dictionary


### Comments on above

#####  `data` is a vector of integers representing the text.
For example

    data[:10]
    [5234, 3081, 12, 6, 195, 2, 3134, 46, 59, 156]
    
##### `count`     is an array of word count tuples
Where the count is the number of times that word appeared in the text

    [['UNK', 1737307],
     ('the', 1061396),
     ('of', 593677),
     ('and', 416629),
     ('one', 411764),
     ('in', 372201),
     
##### `dictionary` is the mapping (a Python dictionary) from words to their associated integers

    {'UNK': 0,
     'the': 1,
     'of': 2,
     'and': 3,
     'one': 4,
     'in': 5,
     'a': 6,
     
     
##### `reverse_dictionary` is the mapping (a Python dictionary) from integers to their associated words

     reverse_dictionary[5234]
     'anarchism'

In [4]:
vocab_size = 25
data, count, dictionary, reverse_dictionary = collect_data(vocabulary_size=vocab_size)
print(data[:7])

window_size = 1
vector_dim = 4
epochs = 200000

valid_size = 16     # Random set of words to evaluate similarity on.
valid_window = 100  # Only pick dev samples in the head of the distribution.
valid_examples = np.random.choice(valid_window, valid_size, replace=False)

sampling_table = sequence.make_sampling_table(vocab_size)
print("Making skipgrams")
couples, labels = skipgrams(data, vocab_size, window_size=window_size, sampling_table=sampling_table)
print("Skipgrams complete")
word_target, word_context = zip(*couples)
word_target = np.array(word_target, dtype="int32")
word_context = np.array(word_context, dtype="int32")

print(couples[:10], labels[:10])
print("Done")

['the', 'lazy', 'old', 'dog', 'sat', 'on', 'the']
[1, 2, 3, 11, 4, 5, 1]
Making skipgrams
Skipgrams complete
[[2, 3], [16, 3], [16, 7], [9, 1], [16, 20], [19, 4], [16, 4], [9, 9], [9, 1], [9, 10]] [1, 1, 0, 1, 0, 1, 1, 0, 1, 0]
Done


In [6]:
#len(couples)
def printPair(e):
    pair = e[0]
    label = e[1]
    print(reverse_dictionary[pair[0]], reverse_dictionary[pair[1]], label)
    
for couple in zip(couples, labels):    
    printPair(couple)
#for w in data:
#    print(reverse_dictionary[w])
max(data)


lazy old 1
burro old 1
burro chair. 0
mat. the 1
burro elephant 0
bear sat 1
burro sat 1
mat. mat. 0
mat. the 1
mat. floor. 0
bear old 1
lazy burro 0
bear pig 0
lazy pig 0
lazy the 1
bear on 0


23

In [7]:
import keras.layers
# create some input variables
input_target = Input((1,))
input_context = Input((1,))

embedding = Embedding(vocab_size, vector_dim, input_length=1, name='embedding')
target = embedding(input_target)
target = Reshape((vector_dim, 1))(target)
context = embedding(input_context)
context = Reshape((vector_dim, 1))(context)





## Setup Cosine Similarity Operation
 which will be output in a secondary model
 
 ```similarity = keras.layers.merge([target, context], mode='cos', dot_axes=0)```

In [8]:
from keras import backend as K

def cosine_distance(vests):
    x, y = vests
    x = K.l2_normalize(x, axis=-1)
    y = K.l2_normalize(y, axis=-1)
    return -K.mean(x * y, axis=-1, keepdims=True)

def cos_dist_output_shape(shapes):
    shape1, shape2 = shapes
    return (shape1[0],1)

#similarity = Lambda(cosine_distance, output_shape=cos_dist_output_shape)([target, context])
similarity =  dot([target, context], 1, normalize=True)

In [9]:

# now perform the dot product operation to get a similarity measure
dot_product = dot([target, context],  1)
dot_product = Reshape((1,))(dot_product)
# add the sigmoid output layer
output = Dense(1, activation='sigmoid')(dot_product)
# create the primary training model
model = Model(input=[input_target, input_context], output=output)
model.compile(loss='binary_crossentropy', optimizer='rmsprop')

# create a secondary validation model to run our similarity checks during training
validation_model = Model(input=[input_target, input_context], output=similarity)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=[<tf.Tenso..., outputs=Tensor("de...)`
  import sys
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:11: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=[<tf.Tenso..., outputs=Tensor("do...)`
  # This is added back by InteractiveShellApp.init_path()


In [10]:
similarity =  dot([target, context], 1, normalize=True)


class SimilarityCallback:
    def run_sim(self):
        for i in range(valid_size):
            valid_word = reverse_dictionary[valid_examples[i]]
            top_k = 8  # number of nearest neighbors
            sim = self._get_sim(valid_examples[i])
            nearest = (-sim).argsort()[1:top_k + 1]
            log_str = 'Nearest to %s:' % valid_word
            for k in range(top_k):
                close_word = reverse_dictionary[nearest[k]]
                log_str = '%s %s,' % (log_str, close_word)
            print(log_str)

    @staticmethod
    def _get_sim(valid_word_idx):
        sim = np.zeros((vocab_size,))
        in_arr1 = np.zeros((1,))
        in_arr2 = np.zeros((1,))
        in_arr1[0,] = valid_word_idx
        for i in range(vocab_size):
            in_arr2[0,] = i
            out = validation_model.predict_on_batch([in_arr1, in_arr2])
            sim[i] = out
        return sim
    
    def get_sim(self, valid_word_idx):
        sim = np.zeros((vocab_size,))
        in_arr1 = np.zeros((1,))
        in_arr2 = np.zeros((1,))
        in_arr1[0,] = valid_word_idx
        for i in range(vocab_size):
            in_arr2[0,] = i
            out = validation_model.predict_on_batch([in_arr1, in_arr2])
            sim[i] = out
        return sim
    
      
    
    
sim_cb = SimilarityCallback()

In [11]:
epochs = 10000

arr_1 = np.zeros((1,))
arr_2 = np.zeros((1,))
arr_3 = np.zeros((1,))
for cnt in range(epochs):
    idx = np.random.randint(0, len(labels)-1)
    arr_1[0,] = word_target[idx]
    arr_2[0,] = word_context[idx]
    arr_3[0,] = labels[idx]
    loss = model.train_on_batch([arr_1, arr_2], arr_3)
    if cnt % 5 == 0:
        print("Iteration {}, loss={}".format(cnt, loss))
    #if cnt % 10000 == 0:
    #    sim_cb.run_sim()

/usr/local/lib/python3.6/dist-packages/tensorflow_core/python/framework/indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Iteration 0, loss=0.6917778253555298
Iteration 5, loss=0.6975952386856079
Iteration 10, loss=0.6879696846008301
Iteration 15, loss=0.6968216300010681
Iteration 20, loss=0.6965695023536682
Iteration 25, loss=0.6908119320869446
Iteration 30, loss=0.6966639757156372
Iteration 35, loss=0.6891825199127197
Iteration 40, loss=0.6891002058982849
Iteration 45, loss=0.6908366680145264
Iteration 50, loss=0.68485426902771
Iteration 55, loss=0.6903255581855774
Iteration 60, loss=0.6906092166900635
Iteration 65, loss=0.6833970546722412
Iteration 70, loss=0.6859609484672546
Iteration 75, loss=0.6950205564498901
Iteration 80, loss=0.6930654048919678
Iteration 85, loss=0.6875976920127869
Iteration 90, loss=0.6858574151992798
Iteration 95, loss=0.693886399269104
Iteration 100, loss=0.6885250210762024
Iteration 105, loss=0.6875311732292175
Iteration 110, loss=0.6866088509559631
Iteration 115, loss=0.6850278377532959
Iteration 120, loss=0.6863799095153809
Iteration 125, loss=0.682975172996521
Iteration 13

Iteration 1070, loss=0.3674347400665283
Iteration 1075, loss=0.4104231595993042
Iteration 1080, loss=0.28318876028060913
Iteration 1085, loss=0.20010925829410553
Iteration 1090, loss=0.38863763213157654
Iteration 1095, loss=0.19473513960838318
Iteration 1100, loss=0.18988005816936493
Iteration 1105, loss=0.2636026442050934
Iteration 1110, loss=0.34390079975128174
Iteration 1115, loss=0.33191201090812683
Iteration 1120, loss=0.18064498901367188
Iteration 1125, loss=0.38855165243148804
Iteration 1130, loss=0.3719998896121979
Iteration 1135, loss=0.10916591435670853
Iteration 1140, loss=0.10648361593484879
Iteration 1145, loss=0.2628527283668518
Iteration 1150, loss=0.30662405490875244
Iteration 1155, loss=0.31286007165908813
Iteration 1160, loss=0.3689816892147064
Iteration 1165, loss=0.22294072806835175
Iteration 1170, loss=0.09709243476390839
Iteration 1175, loss=0.30207955837249756
Iteration 1180, loss=0.34604397416114807
Iteration 1185, loss=0.23752671480178833
Iteration 1190, loss=0

Iteration 2110, loss=0.0001319493749178946
Iteration 2115, loss=0.028005508705973625
Iteration 2120, loss=0.00011419018119340762
Iteration 2125, loss=0.03132152929902077
Iteration 2130, loss=0.002758654300123453
Iteration 2135, loss=0.00010759483848232776
Iteration 2140, loss=0.0038249718490988016
Iteration 2145, loss=0.0013627787120640278
Iteration 2150, loss=0.015629373490810394
Iteration 2155, loss=0.0013008840614929795
Iteration 2160, loss=0.02133755385875702
Iteration 2165, loss=0.026615971699357033
Iteration 2170, loss=0.025433529168367386
Iteration 2175, loss=0.01397379394620657
Iteration 2180, loss=0.013317315839231014
Iteration 2185, loss=0.004995299503207207
Iteration 2190, loss=0.0021243442315608263
Iteration 2195, loss=0.001046987366862595
Iteration 2200, loss=0.021418824791908264
Iteration 2205, loss=0.000940675032325089
Iteration 2210, loss=0.004109518602490425
Iteration 2215, loss=0.016099821776151657
Iteration 2220, loss=0.004322447348386049
Iteration 2225, loss=0.00046

Iteration 3085, loss=6.642634531317526e-08
Iteration 3090, loss=3.681460293591954e-05
Iteration 3095, loss=9.280561789637432e-05
Iteration 3100, loss=6.886130722705275e-05
Iteration 3105, loss=3.4331293136347085e-05
Iteration 3110, loss=7.305317808459222e-07
Iteration 3115, loss=4.6858609437094856e-08
Iteration 3120, loss=8.087938476819545e-05
Iteration 3125, loss=9.355732686344709e-07
Iteration 3130, loss=3.0590610549552366e-05
Iteration 3135, loss=2.4790406314423308e-06
Iteration 3140, loss=4.055390334656295e-08
Iteration 3145, loss=0.0002849665470421314
Iteration 3150, loss=7.438752618327271e-07
Iteration 3155, loss=5.686108852387406e-05
Iteration 3160, loss=2.5849269150057808e-05
Iteration 3165, loss=2.3673122996115126e-05
Iteration 3170, loss=1.639107978235188e-07
Iteration 3175, loss=1.5689045085309772e-07
Iteration 3180, loss=1.4476476906111202e-07
Iteration 3185, loss=0.00011187503696419299
Iteration 3190, loss=4.0832989611772064e-07
Iteration 3195, loss=9.792298078536987e-05
I

...

Iteration 5950, loss=2.333367187822336e-16
Iteration 5955, loss=3.776938761745896e-09
Iteration 5960, loss=2.9816675795046876e-09
Iteration 5965, loss=8.975427345792042e-11
Iteration 5970, loss=2.6411022258088224e-09
Iteration 5975, loss=2.9706948012631074e-09
Iteration 5980, loss=8.951406976764886e-11
Iteration 5985, loss=5.912169083292784e-13
Iteration 5990, loss=9.267550106235173e-14
Iteration 5995, loss=1.9056164646258367e-08
Iteration 6000, loss=3.6557037397244585e-09
Iteration 6005, loss=2.6315030154933083e-09
Iteration 6010, loss=2.2476868935318074e-16
Iteration 6015, loss=1.6313021766745095e-10
Iteration 6020, loss=8.88884729710604e-11
Iteration 6025, loss=9.132750572503692e-14
Iteration 6030, loss=2.208673022252583e-09
Iteration 6035, loss=2.1958113105569055e-09
Iteration 6040, loss=2.1832493590778768e-09
Iteration 6045, loss=1.5996980129440175e-10
Iteration 6050, loss=8.818126090437417e-11
Iteration 6055, loss=2.165131629539019e-09
Iteration 6060, loss=4.43285319562392e-09
It

Iteration 6955, loss=2.5415369808712285e-09
Iteration 6960, loss=1.4081847909963585e-09
Iteration 6965, loss=2.6979336542609644e-09
Iteration 6970, loss=1.505113900179822e-08
Iteration 6975, loss=1.1147156114832413e-10
Iteration 6980, loss=3.292708994351301e-09
Iteration 6985, loss=2.04150074623044e-09
Iteration 6990, loss=3.14866161572544e-13
Iteration 6995, loss=2.037470636651051e-09
Iteration 7000, loss=1.3837798684690483e-09
Iteration 7005, loss=1.4978253304320788e-08
Iteration 7010, loss=3.2824127860209273e-09
Iteration 7015, loss=2.024829637292669e-09
Iteration 7020, loss=6.313318059525744e-14
Iteration 7025, loss=2.017682021460132e-09
Iteration 7030, loss=2.0699431058091022e-09
Iteration 7035, loss=3.2544162920089548e-09
Iteration 7040, loss=1.361965207280491e-09
Iteration 7045, loss=1.3567174050876929e-09
Iteration 7050, loss=2.0087886909436747e-09
Iteration 7055, loss=1.3425337508365942e-09
Iteration 7060, loss=3.0452168022464254e-13
Iteration 7065, loss=7.327709966586937e-11


Iteration 7925, loss=2.1997514758082676e-13
Iteration 7930, loss=2.7340818498089448e-09
Iteration 7935, loss=1.8040113847206385e-09
Iteration 7940, loss=2.727997605589394e-09
Iteration 7945, loss=1.5311643064563896e-09
Iteration 7950, loss=1.0186590460037337e-09
Iteration 7955, loss=1.061936427682042e-09
Iteration 7960, loss=5.036152972513058e-14
Iteration 7965, loss=1.3021004718893892e-08
Iteration 7970, loss=1.778961866705231e-09
Iteration 7975, loss=8.775458831822291e-11
Iteration 7980, loss=1.010240446852606e-09
Iteration 7985, loss=1.0070007050444474e-09
Iteration 7990, loss=5.0103892795761926e-14
Iteration 7995, loss=7.261737864775245e-17
Iteration 8000, loss=1.727721743449706e-09
Iteration 8005, loss=1.046645881075392e-09
Iteration 8010, loss=1.725930065532566e-09
Iteration 8015, loss=1.5044299139788109e-09
Iteration 8020, loss=1.7245183059344527e-09
Iteration 8025, loss=4.976331440019813e-14
Iteration 8030, loss=1.7245183059344527e-09
Iteration 8035, loss=4.971549769625973e-14


Iteration 8885, loss=1.1897530960425229e-08
Iteration 8890, loss=4.2288872056928106e-14
Iteration 8895, loss=2.4021791222850197e-09
Iteration 8900, loss=1.5678246478856295e-09
Iteration 8905, loss=5.632323745199299e-17
Iteration 8910, loss=1.415431216678087e-09
Iteration 8915, loss=5.615439996280356e-17
Iteration 8920, loss=2.1432893237971484e-09
Iteration 8925, loss=4.470682071072707e-12
Iteration 8930, loss=8.21788082028263e-10
Iteration 8935, loss=8.198515200064094e-10
Iteration 8940, loss=1.5643715212121378e-09
Iteration 8945, loss=2.140053911858786e-09
Iteration 8950, loss=7.476727570399078e-11
Iteration 8955, loss=7.475457752814663e-11
Iteration 8960, loss=1.6584338611604077e-13
Iteration 8965, loss=1.1828497292754037e-08
Iteration 8970, loss=4.452522552045313e-12
Iteration 8975, loss=2.387858133445775e-09
Iteration 8980, loss=2.387858133445775e-09
Iteration 8985, loss=1.3987327962539098e-09
Iteration 8990, loss=1.6484682813667356e-13
Iteration 8995, loss=1.6476478113727072e-13
I

Iteration 9840, loss=3.993948834851713e-12
Iteration 9845, loss=5.688374077328362e-11
Iteration 9850, loss=1.135419203102117e-09
Iteration 9855, loss=6.895261028816435e-10
Iteration 9860, loss=6.880219172167301e-10
Iteration 9865, loss=6.638194993691116e-10
Iteration 9870, loss=2.0219041996227816e-09
Iteration 9875, loss=1.0223534241404764e-09
Iteration 9880, loss=6.589513934507352e-10
Iteration 9885, loss=3.962089337172792e-12
Iteration 9890, loss=4.38473553463334e-17
Iteration 9895, loss=5.664781144165687e-11
Iteration 9900, loss=6.570362587332568e-10
Iteration 9905, loss=2.132471088600596e-09
Iteration 9910, loss=1.3252207220048468e-13
Iteration 9915, loss=1.0870682132235743e-08
Iteration 9920, loss=1.4249417201739334e-09
Iteration 9925, loss=6.766746052377925e-10
Iteration 9930, loss=6.476655323162106e-11
Iteration 9935, loss=1.4231680278697922e-09
Iteration 9940, loss=5.648608317199155e-11
Iteration 9945, loss=2.1269344063767903e-09
Iteration 9950, loss=6.463302809622817e-11
Itera

In [ ]:
sim_cb.run_sim()

In [ ]:
valid_examples = np.array( [6, 7, 8, 11, 12, 13, 14])

In [ ]:
data[:10]

In [12]:
def compare(w1, w2):
    in_arr1 = np.zeros((1,))
    in_arr2 = np.zeros((1,))
    in_arr1[0,] = dictionary[w1]
    in_arr2[0,] = dictionary[w2]
    print(validation_model.predict_on_batch([in_arr1, in_arr2]))

In [15]:
compare('frog', 'dog')

[[[-0.39483848]]]


In [ ]:
valid_examples

In [16]:
valid = dictionary['dog']
in_arr1 = np.zeros((1,))
in_arr2 = np.zeros((1,))
in_arr3 = np.zeros((1,))
in_arr1[0,] = valid
in_arr2[0,] = 12
in_arr3[0,] = dictionary["mat."]
#x  = sim_cb.get_sim(valid)

In [17]:
validation_model.predict_on_batch([in_arr1, in_arr2])

array([[[0.90473485]]], dtype=float32)

In [20]:
validation_model.predict_on_batch([in_arr1, in_arr3])

array([[[0.91498697]]], dtype=float32)

In [19]:
reverse_dictionary[12]

'cat'

In [23]:
embedding.get_weights()

[array([[-2.0180428e-02,  3.8796391e-02,  3.9581601e-02, -1.7700054e-02],
        [-1.3642136e+00,  1.3702832e+00, -3.4327168e-02,  3.5242200e-01],
        [ 1.6238487e+00, -1.6949019e+00, -6.9046557e-01,  2.4016201e-01],
        [-1.1019892e+00,  1.1608373e+00,  1.1630919e+00, -1.0825593e+00],
        [-9.2466658e-01,  9.9118060e-01,  9.4729191e-01, -9.7021735e-01],
        [ 2.5630962e-02,  4.9725462e-02, -1.7837264e-02,  4.6070624e-02],
        [-1.3387740e-02,  4.9413089e-02, -4.6420097e-03, -3.5513185e-02],
        [ 7.5155848e-01, -7.2349083e-01, -7.9123420e-01,  7.6966560e-01],
        [ 3.4375515e-02, -1.4950313e-02, -8.1053749e-03, -3.0967152e-02],
        [ 1.5945641e+00, -1.5981613e+00,  1.3569762e+00, -1.4269550e+00],
        [ 7.9121387e-01, -8.3454752e-01,  8.4425145e-01, -8.0594784e-01],
        [ 3.0069660e-02, -2.2222495e-02,  1.3444815e-02, -4.7114935e-02],
        [ 4.3911446e-02, -2.2283113e-02,  1.9387674e-02, -2.5411725e-02],
        [-1.4537644e-02,  6.6388957e-0